### Importando as bibliotecas

In [2]:
#import libraries
import numpy as np

In [3]:
# Modelando ambiente como figura do cliff walk:
environment_rows = 4
environment_columns = 9
num_actions = 4 #up right down left

In [4]:
# Criando a tabela Q(s,a)
q_values = np.zeros((environment_rows, environment_columns, num_actions))

In [5]:
# Definindo as ações numericamente: 0 = up, 1 = right, 2 = down, 3 = left
actions = ['up', 'right', 'down', 'left']

#### Rewards
As recompensas serão definidas conforme a figura abaixo. Usaremos recompensas negativas (punições) para evitar que o agente ande em círculos e nunca encontre o destino desejado. Além disso, punições fará com que o agente tente encontrar o menor caminho possível, reduzindo, assim, o valor das punições (maximizando as recompensas).
![08-warehouse-map-rewards.png](attachment:08-warehouse-map-rewards.png)


In [7]:
# Criando a matriz 2D para armazenas o valor das recompensas de cada estado
rewards = np.full((environment_rows,environment_columns),-1.)
rewards[3,8] = 100
rewards[3,1] = -100
rewards[3,2] = -100
rewards[3,3] = -100
rewards[3,4] = -100
rewards[3,5] = -100
rewards[3,6] = -100
rewards[3,7] = -100
rewards[3,8] = 100


  
#print rewards matrix
for row in rewards:
  print(row)


[-1. -1. -1. -1. -1. -1. -1. -1. -1.]
[-1. -1. -1. -1. -1. -1. -1. -1. -1.]
[-1. -1. -1. -1. -1. -1. -1. -1. -1.]
[  -1. -100. -100. -100. -100. -100. -100. -100.  100.]


#### Definindo algumas funções auxiliares

In [8]:
def is_terminal_state(current_row_index, current_column_index):
    """
    Função que avalia se a localização é um estado terminal. Se a recompensa for -1, é um estado válido.
    """
    
    if rewards[current_row_index, current_column_index] == -1.:
        return False
    else:
        return True

#define a function that will choose a random, non-terminal starting location
def get_starting_location():
    """
    Função que escolhe o estado inicial de cada época de treinamento de maneira aleatória.
    """
    # escolhe uma linha e uma coluna aleatória
    current_row_index = np.random.randint(environment_rows)
    current_column_index = np.random.randint(environment_columns)
    
    # Verifica se é um estado válido (quadrado branco).
    while is_terminal_state(current_row_index, current_column_index):
        current_row_index = np.random.randint(environment_rows)
        current_column_index = np.random.randint(environment_columns)

    return current_row_index, current_column_index

#define an epsilon greedy algorithm that will choose which action to take next (i.e., where to move next)
def get_next_action(current_row_index, current_column_index, epsilon):
    """
    Escolhe uma ação com base no algoritmo epsilon greedy.
    """
    
    # Se a escolha randomica entre 0 e 1 for menor que epsilon, escolhe o estado com valor mais promissor
    if np.random.random() < epsilon:
        return np.argmax(q_values[current_row_index, current_column_index])
    else: 
        # seleciona uma ação aleatória
        return np.random.randint(4)

#define a function that will get the next location based on the chosen action
def get_next_location(current_row_index, current_column_index, action_index):
    """
    Função que define a próxima localização com base na ação selecionada.
    """
    
    new_row_index = current_row_index
    new_column_index = current_column_index
    
    if actions[action_index] == 'up' and current_row_index > 0:
        new_row_index -= 1
    elif actions[action_index] == 'right' and current_column_index < environment_columns - 1:
        new_column_index += 1
    elif actions[action_index] == 'down' and current_row_index < environment_rows - 1:
        new_row_index += 1
    elif actions[action_index] == 'left' and current_column_index > 0:
        new_column_index -= 1
    
    return new_row_index, new_column_index

def get_shortest_path(start_row_index, start_column_index):
    """
    Função que retorna o menor caminho entre qualquer localização e a área de entrega.
    """
    # Retorna vazio se for uma localização inválida
    if is_terminal_state(start_row_index, start_column_index):
        return []
  
    else: 
        # para localizaçõe válidas
        current_row_index, current_column_index = start_row_index, start_column_index
        shortest_path = []
        shortest_path.append([current_row_index, current_column_index])

    # continua se movendo ao longo do caminho até encontrar o objetivo
    while not is_terminal_state(current_row_index, current_column_index):
        # escolhe a melhor ação 
        action_index = get_next_action(current_row_index, current_column_index, 1.)
        
        # move até a próxima localização no caminho, adicionando a nova localização à lista
        current_row_index, current_column_index = get_next_location(current_row_index, current_column_index, action_index)
        shortest_path.append([current_row_index, current_column_index])

    return shortest_path

In [ ]:
path = get_shortest_path(3,0)
print('Caminho até a área de entrega:\n',path)

#### Training Agent model

In [ ]:
# Definindo os parâmetros de treinamento
epsilon = 0.9 # percentual de vezes que irá escolher a melhor ação ao invés de uma ação aleatória
discount_factor = 0.9 # fator de desconteo para recompensas futuras
learning_rate = 0.9 # taxa de aprendizado
epochs = 1000

# Realizando treinamento durante 1000 episódios
for episode in range(epochs):
    # pega localização inicial para o episódio
    row_index, column_index = get_starting_location()

    # toma ações até que chegue a um estado terminal (área de entrega ou colisão com prateleiras)
    while not is_terminal_state(row_index, column_index):
        # escolhe uma ação
        action_index = get_next_action(row_index, column_index, epsilon)

        # realiza a ação, fazendo a transição para o próximo estado (vai para o próximo estado)
        old_row_index, old_column_index = row_index, column_index # armazena a posição anterior
        row_index, column_index = get_next_location(row_index, column_index, action_index)

        # recebe a recompensa por de deslocar para o novo estado e calcula a TD
        reward = rewards[row_index, column_index]
        old_q_value = q_values[old_row_index, old_column_index, action_index]
        temporal_difference = reward + (discount_factor * np.max(q_values[row_index, column_index])) - old_q_value

        # atualiza a tabela Q para o par estado-ação anterior
        new_q_value = old_q_value + (learning_rate * temporal_difference)
        q_values[old_row_index, old_column_index, action_index] = new_q_value

print('Training complete!')

## Selecionando o menor caminho
Agora que o agente já foi completamente treinado, podemos verificar a qualidade do treinamento, verificando se o caminho sugerido é o menor, se chega até a área de entrega e se não há colisão com as prateleiras.

In [ ]:
# Testado para a posição inicial
print('Origem:\nLinha: 3 / Coluna: 0\n',get_shortest_path(3,0)) # origem: linha 3, coluna 0
